## 규칙
- 각 이름은 띄어쓰기 없이 작성하기. 예를 들어, "영업 시간"이 아니라 "영업시간"으로

In [1]:
from tqdm import tqdm
from langchain_community.vectorstores import FAISS
import pickle
import pandas as pd
import numpy as np
from langchain.schema import Document
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain_community.document_loaders import DataFrameLoader
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

In [144]:
class CFG:
    # store="프랭크버거"
    output_path = "/home/user09/beaver/data/db"
    data_path = ""
    save_path = ""
    embedding_model= "nlpai-lab/KoE5" # "BAAI/bge-m3"
    retriever_k=5
    retriever_bert_weight=0.7
    version='4_general'
    # info_type='STORE_INFO'   # STORE_INFO, TIME_INFO, MENU_INFO
    store_num = 104933  # 102496, 102506, 103807, 104570, 104933
    fill_nan = "정보없음"
    basic_info = ["주차장", "씨씨티비", "영업시간", "예약가능여부", "전화번호"]
    seed=42


CFG.data_path = f"/home/user09/beaver/data/shared_files/dataset/dataset_v{CFG.version}.xlsx"
print(CFG.data_path)
CFG.save_path = f"/home/user09/beaver/data/shared_files/dataset/dataset_v{CFG.version}_{CFG.store_num}_preprocessed.xlsx"
print(CFG.save_path)

/home/user09/beaver/data/shared_files/dataset/dataset_v3_general.xlsx
/home/user09/beaver/data/shared_files/dataset/dataset_v3_general_104933_preprocessed.xlsx


### STORE_INFO

In [145]:
################ 전처리 ################
from langchain.schema import Document

# columns_to_use = ['STORE_NO', 'STORE_NM', 'ADDR', 'DETAIL_ADDR', 'TEL', 'GUID_CONTS', 'PAYMNT_MN_CD']
columns_to_use = ['STORE_NO', 'STORE_NM', 'ADDR', 'DETAIL_ADDR', 'TEL', 'PAYMNT_MN_CD']  # 'GUID_CONTS'
df_store = pd.read_excel(CFG.data_path, sheet_name='STORE_INFO', usecols=columns_to_use)

df_store = df_store[df_store['STORE_NO']==CFG.store_num]  # 해당 매장 정보만 가져옴
store = df_store['STORE_NM'].unique()[0]
print(store)
df_store.drop(columns=['STORE_NO', 'STORE_NM'], inplace=True)

# for col in df_store.columns:   # 매장 정보는 매장 당 한 개의 row만 존재하므로 제공해주지 않는 매장 정보 컬럼은 삭제
#     if df_store[col].isnull().values.any():   
#         df_store.drop(columns=[col], inplace=True)

# df_2.drop(columns=['STORE_NO'], inplace=True)
# print(df_2)

# 1. 주소와 상세 주소 합치기
df_store['ADDR'] = df_store['ADDR'] + ' ' + df_store['DETAIL_ADDR']
df_store['ADDR'] = df_store['ADDR'].fillna(CFG.fill_nan)   # !!!!!!!!!!!!!!!
df_store.drop(columns=['DETAIL_ADDR'], inplace=True)

df_store.fillna(CFG.fill_nan, inplace=True)  # !!!!!!!!!!!!!!!

# 2. 결제수단 매핑
def map_payment_methods(payment_codes):
    payment_mapping = {
    'CA': '현금',
    'CD': '신용카드',
    'PO': '포인트',
    'PC': '간편결제',
    'GV': '모바일상품권',
    'ET': '기타결제'
    }
    codes = payment_codes.split(', ')
    mapped_names = [payment_mapping[code] for code in codes if code in payment_mapping]
    return ', '.join(mapped_names)

df_store['PAYMNT_MN_CD'] = df_store['PAYMNT_MN_CD'].apply(map_payment_methods)
df_store_col = df_store.columns

# 결과 출력
print(df_store)
print(df_store_col)

홍콩반점0410 선릉역점
                             ADDR   TEL            PAYMNT_MN_CD
4  서울 강남구 선릉로76길 22 ,2층(대치동,우리빌딩)  정보없음  현금, 신용카드, 기타결제, 모바일상품권
Index(['ADDR', 'TEL', 'PAYMNT_MN_CD'], dtype='object')


In [146]:
############## 컬럼명 변경 ##############
store_column_mapping = {
    'STORE_NO': '상점번호',
    'STORE_NM': '상점명',
    'ADDR': '위치',
    'DETAIL_ADDR': '매장주소',
    'TEL': '전화번호',
    'X': '좌표 X',
    'Y': '좌표 Y',
    'GUID_TTL': '안내제목',  # !!!!!!!!!!!!!!!!!
    'GUID_CONTS': '안내내용',
    'PAYMNT_MN_CD': '결제수단',
}

for col in df_store_col:
    df_store = df_store.rename(columns={col: store_column_mapping[col]})
    
print(df_store)

                               위치  전화번호                    결제수단
4  서울 강남구 선릉로76길 22 ,2층(대치동,우리빌딩)  정보없음  현금, 신용카드, 기타결제, 모바일상품권


In [147]:
############### 한 컬럼으로 통합 ###############
def create_contents_rowwise(df):
    contents_data = []
    for _, row in df.iterrows():
        for col in df.columns:
            contents_data.append(f"{col}: {row[col]}")
    new_df = pd.DataFrame({'contents': contents_data})
    return new_df

# 함수 호출
result_df_store = create_contents_rowwise(df_store)
print(result_df_store)

                             contents
0  위치: 서울 강남구 선릉로76길 22 ,2층(대치동,우리빌딩)
1                          전화번호: 정보없음
2        결제수단: 현금, 신용카드, 기타결제, 모바일상품권


### MENU_INFO

In [148]:
################ 전처리 ################
columns_to_use = ['STORE_NO', 'PROD_NM', 'PRICE']
df_menu = pd.read_excel(f'/home/user09/beaver/data/shared_files/dataset/dataset_v{CFG.version}.xlsx', sheet_name='MENU_INFO', usecols=columns_to_use)
df_menu = df_menu[df_menu['STORE_NO']==CFG.store_num]
df_menu.drop(columns=['STORE_NO'], inplace=True)

df_menu_col = df_menu.columns

print(df_menu[:3])
print(df_menu_col)

      PROD_NM  PRICE
302  연태고량주(중)  24000
303  연태고량주(소)  14000
304       칭따오   7500
Index(['PROD_NM', 'PRICE'], dtype='object')


In [149]:
############## 컬럼명 변경 ##############
menu_column_mapping = {
    # 'PROD_NO': '상품 번호',
    'PROD_NM': '메뉴명',
    'PRICE': '가격'
}

for col in df_menu_col:
    df_menu = df_menu.rename(columns={col: menu_column_mapping[col]})

df_menu['가격'] = df_menu['가격'].astype(str) + '원'
    
print(df_menu[:3])

          메뉴명      가격
302  연태고량주(중)  24000원
303  연태고량주(소)  14000원
304       칭따오   7500원


In [150]:
############### 한 컬럼으로 통합 ###############
def create_contents_rowwise(df):
    contents_data = []
    for _, row in df.iterrows():
        row_content = ", ".join([f"{col}: {row[col]}" for col in df.columns])  # 각 행의 컬럼명:값을 ", "로 결합하여 하나의 문자열 생성
        contents_data.append(row_content)
    
    new_df = pd.DataFrame({'contents': contents_data})  # contents 열을 포함하는 새로운 데이터프레임 생성
    return new_df

# 함수 호출
result_df_menu = create_contents_rowwise(df_menu)
print(result_df_menu[:3])

                    contents
0  메뉴명: 연태고량주(중), 가격: 24000원
1  메뉴명: 연태고량주(소), 가격: 14000원
2        메뉴명: 칭따오, 가격: 7500원


### TIME_INFO

In [151]:
### 전처리 ###
columns_to_use = ['STORE_NO', 'WKDY', 'SALE_BGN_TM', 'SALE_END_TM', 'STORE_REST_BGN_TM', 'STORE_REST_END_TM']
df_time = pd.read_excel(f'/home/user09/beaver/data/shared_files/dataset/dataset_v{CFG.version}.xlsx', sheet_name='TIME_INFO', usecols=columns_to_use)
df_time = df_time[df_time['STORE_NO']==CFG.store_num]

# 1. 요일 매핑
day_mapping = {
    2: '월요일',
    3: '화요일',
    4: '수요일',
    5: '목요일',
    6: '금요일',
    7: '토요일',
    1: '일요일'
}
df_time['WKDY'] = df_time['WKDY'].map(day_mapping)


# 2. 시간 포맷 변경
def convert_time_format(hhmmss):
    if pd.isna(hhmmss): 
        return None
    # if not hhmmss:
    #     return hhmmss
    # hhmmss가 숫자인 경우 문자열로 변환
    hhmmss = str(hhmmss).zfill(6)  # zfill을 사용하여 6자리를 맞춤 (예: 930 -> '000930')
    return hhmmss[:2] + ':' + hhmmss[2:4]

df_time['SALE_BGN_TM'] = df_time['SALE_BGN_TM'].apply(convert_time_format)
df_time['SALE_END_TM'] = df_time['SALE_END_TM'].apply(convert_time_format)
df_time['STORE_REST_BGN_TM'] = df_time['STORE_REST_BGN_TM'].apply(convert_time_format)
df_time['STORE_REST_END_TM'] = df_time['STORE_REST_END_TM'].apply(convert_time_format)

df_time.drop(columns=['STORE_NO'], inplace=True)

print(df_time)

   WKDY SALE_BGN_TM SALE_END_TM STORE_REST_BGN_TM STORE_REST_END_TM
16  NaN        None        None              None              None


In [152]:
import pandas as pd

# SALE_TM 컬럼 생성
df_time['SALE_TM'] = df_time['WKDY'] + ' ' + df_time['SALE_BGN_TM'] + '-' + df_time['SALE_END_TM']
invalid_values = ['00:0n', '00:00', None]  # 잘못된 값들

# 잘못된 값일 경우 "브레이크 타임 없음" 또는 "정보 없음"으로 처리
for col in ['STORE_REST_BGN_TM', 'STORE_REST_END_TM']:
    df_time[col] = df_time.apply(
        lambda row: "정보 없음" if pd.isna(row['SALE_BGN_TM']) else
                    ("브레이크 타임 없음" if row[col] in invalid_values else row[col]), 
        axis=1
    )

# REST_TM 컬럼 생성
# 영업시간이 있는 경우 "브레이크 타임 없음", 없는 경우 "정보 없음"
df_time['REST_TM'] = df_time.apply(
    lambda row: CFG.fill_nan if pd.isna(row['SALE_TM']) 
                else ("브레이크 타임 없음" if row['STORE_REST_BGN_TM'] == "브레이크 타임 없음" 
                      else f"{row['STORE_REST_BGN_TM']}-{row['STORE_REST_END_TM']}"),
    axis=1
)

# 필요 없는 컬럼 삭제
df_time.drop(columns=['WKDY', 'SALE_BGN_TM', 'SALE_END_TM', 'STORE_REST_BGN_TM', 'STORE_REST_END_TM'], inplace=True)
df_time_col = df_time.columns
df_time['SALE_TM'].fillna(CFG.fill_nan, inplace=True)

# 결과 출력
print(df_time)
print(df_time_col)


   SALE_TM REST_TM
16    정보없음    정보없음
Index(['SALE_TM', 'REST_TM'], dtype='object')


In [153]:
############## 컬럼명 변경 ##############
time_column_mapping = {
    'WKDY': '영업요일',
    'SALE_TM': '영업시간',
    'REST_TM': '브레이크타임',
    'SALE_BGN_TM': '영업 시작 시간',
    'SALE_END_TM': '영업 마감 시간',
    'STORE_REST_BGN_TM': '브레이크 타임 시작 시간',
    'STORE_REST_END_TM': '브레이크 타임 마감 시간',
}

for col in df_time_col:
    df_time = df_time.rename(columns={col: time_column_mapping[col]})
    
print(df_time)

    영업시간 브레이크타임
16  정보없음   정보없음


In [154]:
############### 한 컬럼으로 통합 ###############
def create_contents_rowwise(df):
    contents_data = []
    for _, row in df.iterrows():
        row_content = ", ".join([f"{col}: {row[col]}" for col in df.columns])  # 각 행의 컬럼명:값을 ", "로 결합하여 하나의 문자열 생성
        contents_data.append(row_content)
    
    new_df = pd.DataFrame({'contents': contents_data})  # contents 열을 포함하는 새로운 데이터프레임 생성
    return new_df

# 함수 호출
result_df_time = create_contents_rowwise(df_time)
print(result_df_time)

                   contents
0  영업시간: 정보없음, 브레이크타임: 정보없음


### 3개 통합 & 피클파일

In [155]:
df = pd.concat([result_df_store, result_df_time, result_df_menu], ignore_index=True)

### 기본 정보 지정

In [156]:
# ":"를 기준으로 분리하여 앞의 키워드를 추출한 후, 존재 여부 확인
existing_info = [content.split(":")[0].strip() for content in df["contents"]]
missing_info = [info for info in CFG.basic_info if info not in existing_info]
for info in missing_info:
    df = pd.concat([df, pd.DataFrame({"contents": [f"{info}: 정보없음"]})], ignore_index=True)

# 결과 출력
df.to_excel(CFG.save_path, index=False)   # 전처리한 엑셀 파일 저장
print(df)

                               contents
0    위치: 서울 강남구 선릉로76길 22 ,2층(대치동,우리빌딩)
1                            전화번호: 정보없음
2          결제수단: 현금, 신용카드, 기타결제, 모바일상품권
3              영업시간: 정보없음, 브레이크타임: 정보없음
4             메뉴명: 연태고량주(중), 가격: 24000원
..                                  ...
145         메뉴명: [15주년행사]탕수육, 가격: 9900원
146                   메뉴명: 린, 가격: 5000원
147                           주차장: 정보없음
148                          씨씨티비: 정보없음
149                        예약가능여부: 정보없음

[150 rows x 1 columns]


In [157]:
#### 엑셀파일 DB화(pickle파일로 변환) ##### Extracting information from the "Details" column to create documents
docs = []
for _, row in df.iterrows():
    details = row['contents']
    # metadata = {}
    
    # # Parse key-value pairs in the "Details" column
    # for detail in details.split('\n'):
    #     if ':' in detail:
    #         key, value = detail.split(':', 1)
            # metadata[key.strip()] = value.strip()
    
    docs.append(Document(page_content=details))#, metadata=metadata))

# Embeddings and vector store setup
encode_kwargs = {'normalize_embeddings': True}
model_kwargs = {'device': 'cpu'}

hf = HuggingFaceEmbeddings(
    model_name=CFG.embedding_model,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

db = FAISS.from_documents(docs, hf)

# Save the FAISS vector store and documents as pickle
db.save_local(f"{CFG.output_path}/{CFG.store_num}_faiss{CFG.version}")

with open(f"{CFG.output_path}/{CFG.store_num}_docs{CFG.version}.pkl", "wb") as f:
    pickle.dump(docs, f)

# Load the FAISS vector store
db = FAISS.load_local(f"/home/user09/beaver/data/db/{CFG.store_num}_faiss{CFG.version}", hf, allow_dangerous_deserialization=True)

# Create retrievers
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": CFG.retriever_k})
bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = CFG.retriever_k

ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever, bm25_retriever],
    weights=[CFG.retriever_bert_weight, 1 - CFG.retriever_bert_weight]
)
